In [1]:
import matplotlib as mpl
import matplotlib.pyplot as plt
%matplotlib inline
import numpy as np
import sklearn
import pandas as pd
import os
import sys
import time
import tensorflow as tf

from tensorflow import keras

print(tf.__version__)
print(sys.version_info)
for module in mpl, np, pd, sklearn, tf, keras:
    print(module.__name__, module.__version__)

2.0.0
sys.version_info(major=3, minor=7, micro=4, releaselevel='final', serial=0)
matplotlib 3.1.1
numpy 1.17.2
pandas 0.25.1
sklearn 0.21.3
tensorflow 2.0.0
tensorflow_core.keras 2.2.4-tf


In [2]:
from sklearn.datasets import fetch_california_housing

housing = fetch_california_housing()
print(housing.DESCR)
print(housing.data.shape)
print(housing.target.shape)

.. _california_housing_dataset:

California Housing dataset
--------------------------

**Data Set Characteristics:**

    :Number of Instances: 20640

    :Number of Attributes: 8 numeric, predictive attributes and the target

    :Attribute Information:
        - MedInc        median income in block
        - HouseAge      median house age in block
        - AveRooms      average number of rooms
        - AveBedrms     average number of bedrooms
        - Population    block population
        - AveOccup      average house occupancy
        - Latitude      house block latitude
        - Longitude     house block longitude

    :Missing Attribute Values: None

This dataset was obtained from the StatLib repository.
http://lib.stat.cmu.edu/datasets/

The target variable is the median house value for California districts.

This dataset was derived from the 1990 U.S. census, using one row per census
block group. A block group is the smallest geographical unit for which the U.S.
Census Bur

In [3]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(
    housing.data, housing.target, random_state = 7)
X_train, X_val, y_train, y_val = train_test_split(
    X_train, y_train, random_state = 11)
print(X_train.shape, y_train.shape)
print(X_val.shape, y_val.shape)
print(X_test.shape, y_test.shape)

(11610, 8) (11610,)
(3870, 8) (3870,)
(5160, 8) (5160,)


In [4]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_val = scaler.transform(X_val)
X_test = scaler.transform(X_test)

In [5]:
mse_keras = keras.metrics.MeanSquaredError()
print(mse_keras([-5],[1]))
print(mse_keras([0.], [1.]))
print(mse_keras.result())

mse_keras.reset_states()
mse_keras([1.], [3.])
print(mse_keras.result())

tf.Tensor(36.0, shape=(), dtype=float32)
tf.Tensor(18.5, shape=(), dtype=float32)
tf.Tensor(18.5, shape=(), dtype=float32)
tf.Tensor(4.0, shape=(), dtype=float32)


In [6]:
def random_batch(x, y, batch_size=32):
    idx = np.random.randint(0, len(x), size=batch_size)
    return x[idx], y[idx]

In [7]:
epochs = 100
batch_size = 32
steps_per_epoch = len(X_train) // batch_size 
optimizer = keras.optimizers.SGD()
metric = keras.metrics.MeanSquaredError()

In [8]:
model = keras.models.Sequential(
    [
        keras.layers.Dense(30,activation='relu',input_shape=[None,X_train.shape[1]])
        ,keras.layers.Dense(1)
    ]
)

In [ ]:
for epoch in range(epochs):
    metric.reset_states() # 
    for step in range(steps_per_epoch): # 每一大步训练多少次 : len / batch_size
        X_batch, y_batch = random_batch(X_train,y_train,batch_size) # random pick batch_size
        with tf.GradientTape() as tape:
            y_pred = model(X_batch) # batch pred
            y_pred = tf.squeeze(y_pred, 1) # vectorized y_pred
            loss = keras.losses.mean_squared_error(y_batch, y_pred) # mean_squared_error
            metric(y_batch, y_pred)
        grads = tape.gradient(loss,model.variables) # get grads
        grads_and_vars = zip(grads,model.variables)
        optimizer.apply_gradients(grads_and_vars) # 这个只是一个计算步长的工具，步长 = learning_rate * grad , 
        print('\nEpoch :' , epoch , 'step : ', step , ' Train mse : ' , metric.result().numpy() , end="")
        
    y_val_pred = model(X_val)
    y_val_pred = tf.squeeze(y_val_pred,1)
    val_loss = keras.losses.mean_squared_error(y_val_pred,y_val)
    print('\n', 'valid mse : ' , val_loss.numpy())


If you intended to run this layer in float32, you can safely ignore this warning. If in doubt, this warning is likely only an issue if you are porting a TensorFlow 1.X model to TensorFlow 2.

To change all layers to have dtype float64 by default, call `tf.keras.backend.set_floatx('float64')`. To change just this layer, pass dtype='float64' to the layer constructor. If you are the author of this layer, you can disable autocasting by passing autocast=False to the base Layer constructor.


Epoch : 0 step :  0  Train mse :  6.303696
Epoch : 0 step :  1  Train mse :  5.009488
Epoch : 0 step :  2  Train mse :  4.3065763
Epoch : 0 step :  3  Train mse :  3.888527
Epoch : 0 step :  4  Train mse :  3.6408963
Epoch : 0 step :  5  Train mse :  3.577912
Epoch : 0 step :  6  Train mse :  3.4367464
Epoch : 0 step :  7  Train mse :  3.366538
Epoch : 0 step :  8  Train mse :  3.3274775
Epoch : 0 step :  9  Train mse :  3.291357
Epoch : 0 step :  10  Train mse :  3.2751353
Epoch : 0 step :  11  Train 

Epoch : 0 step :  165  Train mse :  1.0149206
Epoch : 0 step :  166  Train mse :  1.0111806
Epoch : 0 step :  167  Train mse :  1.0117857
Epoch : 0 step :  168  Train mse :  1.0089183
Epoch : 0 step :  169  Train mse :  1.0068223
Epoch : 0 step :  170  Train mse :  1.005475
Epoch : 0 step :  171  Train mse :  1.0047557
Epoch : 0 step :  172  Train mse :  1.0041386
Epoch : 0 step :  173  Train mse :  1.0007898
Epoch : 0 step :  174  Train mse :  1.0000039
Epoch : 0 step :  175  Train mse :  0.99709535
Epoch : 0 step :  176  Train mse :  0.9961905
Epoch : 0 step :  177  Train mse :  0.99567944
Epoch : 0 step :  178  Train mse :  0.995368
Epoch : 0 step :  179  Train mse :  0.9922738
Epoch : 0 step :  180  Train mse :  0.9909336
Epoch : 0 step :  181  Train mse :  0.9902031
Epoch : 0 step :  182  Train mse :  0.98833156
Epoch : 0 step :  183  Train mse :  0.98920757
Epoch : 0 step :  184  Train mse :  0.9875819
Epoch : 0 step :  185  Train mse :  0.98724633
Epoch : 0 step :  186  Train ms

Epoch : 0 step :  345  Train mse :  0.8050143
Epoch : 0 step :  346  Train mse :  0.80380356
Epoch : 0 step :  347  Train mse :  0.8036043
Epoch : 0 step :  348  Train mse :  0.80198115
Epoch : 0 step :  349  Train mse :  0.8006877
Epoch : 0 step :  350  Train mse :  0.8000271
Epoch : 0 step :  351  Train mse :  0.7996876
Epoch : 0 step :  352  Train mse :  0.7989352
Epoch : 0 step :  353  Train mse :  0.79795337
Epoch : 0 step :  354  Train mse :  0.7970647
Epoch : 0 step :  355  Train mse :  0.79634273
Epoch : 0 step :  356  Train mse :  0.7950454
Epoch : 0 step :  357  Train mse :  0.79371583
Epoch : 0 step :  358  Train mse :  0.7934223
Epoch : 0 step :  359  Train mse :  0.7928943
Epoch : 0 step :  360  Train mse :  0.79229045
Epoch : 0 step :  361  Train mse :  0.7910872
 valid mse :  0.6225541020794079

Epoch : 1 step :  0  Train mse :  0.6872601
Epoch : 1 step :  1  Train mse :  0.50684774
Epoch : 1 step :  2  Train mse :  0.4884614
Epoch : 1 step :  3  Train mse :  0.5086928
E

Epoch : 1 step :  164  Train mse :  0.53735816
Epoch : 1 step :  165  Train mse :  0.54151803
Epoch : 1 step :  166  Train mse :  0.54494804
Epoch : 1 step :  167  Train mse :  0.5448079
Epoch : 1 step :  168  Train mse :  0.5439915
Epoch : 1 step :  169  Train mse :  0.54347605
Epoch : 1 step :  170  Train mse :  0.54175836
Epoch : 1 step :  171  Train mse :  0.5429224
Epoch : 1 step :  172  Train mse :  0.5425702
Epoch : 1 step :  173  Train mse :  0.54216623
Epoch : 1 step :  174  Train mse :  0.54248184
Epoch : 1 step :  175  Train mse :  0.542237
Epoch : 1 step :  176  Train mse :  0.5430124
Epoch : 1 step :  177  Train mse :  0.543903
Epoch : 1 step :  178  Train mse :  0.5451982
Epoch : 1 step :  179  Train mse :  0.5487859
Epoch : 1 step :  180  Train mse :  0.55010617
Epoch : 1 step :  181  Train mse :  0.55024564
Epoch : 1 step :  182  Train mse :  0.5516611
Epoch : 1 step :  183  Train mse :  0.55143756
Epoch : 1 step :  184  Train mse :  0.5524239
Epoch : 1 step :  185  Tra

Epoch : 1 step :  340  Train mse :  0.5700148
Epoch : 1 step :  341  Train mse :  0.5703602
Epoch : 1 step :  342  Train mse :  0.57026386
Epoch : 1 step :  343  Train mse :  0.57015896
Epoch : 1 step :  344  Train mse :  0.56976104
Epoch : 1 step :  345  Train mse :  0.5686573
Epoch : 1 step :  346  Train mse :  0.56829494
Epoch : 1 step :  347  Train mse :  0.56757885
Epoch : 1 step :  348  Train mse :  0.56739753
Epoch : 1 step :  349  Train mse :  0.56640893
Epoch : 1 step :  350  Train mse :  0.5659472
Epoch : 1 step :  351  Train mse :  0.5654379
Epoch : 1 step :  352  Train mse :  0.5647886
Epoch : 1 step :  353  Train mse :  0.5642409
Epoch : 1 step :  354  Train mse :  0.56482947
Epoch : 1 step :  355  Train mse :  0.5639999
Epoch : 1 step :  356  Train mse :  0.5639986
Epoch : 1 step :  357  Train mse :  0.5637287
Epoch : 1 step :  358  Train mse :  0.56320703
Epoch : 1 step :  359  Train mse :  0.56288487
Epoch : 1 step :  360  Train mse :  0.56264347
Epoch : 1 step :  361  

...

Epoch : 18 step :  179  Train mse :  0.36422953
Epoch : 18 step :  180  Train mse :  0.3640028
Epoch : 18 step :  181  Train mse :  0.36437082
Epoch : 18 step :  182  Train mse :  0.36471215
Epoch : 18 step :  183  Train mse :  0.36444727
Epoch : 18 step :  184  Train mse :  0.3674081
Epoch : 18 step :  185  Train mse :  0.36777532
Epoch : 18 step :  186  Train mse :  0.36727256
Epoch : 18 step :  187  Train mse :  0.36688882
Epoch : 18 step :  188  Train mse :  0.3664436
Epoch : 18 step :  189  Train mse :  0.36633557
Epoch : 18 step :  190  Train mse :  0.36707222
Epoch : 18 step :  191  Train mse :  0.3680487
Epoch : 18 step :  192  Train mse :  0.3676468
Epoch : 18 step :  193  Train mse :  0.36816114
Epoch : 18 step :  194  Train mse :  0.36806917
Epoch : 18 step :  195  Train mse :  0.36831143
Epoch : 18 step :  196  Train mse :  0.36821887
Epoch : 18 step :  197  Train mse :  0.3697677
Epoch : 18 step :  198  Train mse :  0.36935604
Epoch : 18 step :  199  Train mse :  0.3697996

Epoch : 18 step :  352  Train mse :  0.38132828
Epoch : 18 step :  353  Train mse :  0.38160485
Epoch : 18 step :  354  Train mse :  0.3824243
Epoch : 18 step :  355  Train mse :  0.3825526
Epoch : 18 step :  356  Train mse :  0.38213214
Epoch : 18 step :  357  Train mse :  0.38266188
Epoch : 18 step :  358  Train mse :  0.38258094
Epoch : 18 step :  359  Train mse :  0.38264477
Epoch : 18 step :  360  Train mse :  0.38260043
Epoch : 18 step :  361  Train mse :  0.38433236
 valid mse :  0.3819363694397655

Epoch : 19 step :  0  Train mse :  0.5199646
Epoch : 19 step :  1  Train mse :  0.42774555
Epoch : 19 step :  2  Train mse :  0.34775653
Epoch : 19 step :  3  Train mse :  0.29925525
Epoch : 19 step :  4  Train mse :  0.30154467
Epoch : 19 step :  5  Train mse :  0.32326457
Epoch : 19 step :  6  Train mse :  0.31104547
Epoch : 19 step :  7  Train mse :  0.30047417
Epoch : 19 step :  8  Train mse :  0.29547018
Epoch : 19 step :  9  Train mse :  0.3909383
Epoch : 19 step :  10  Train m

Epoch : 19 step :  166  Train mse :  0.3705963
Epoch : 19 step :  167  Train mse :  0.36947957
Epoch : 19 step :  168  Train mse :  0.36838862
Epoch : 19 step :  169  Train mse :  0.36835966
Epoch : 19 step :  170  Train mse :  0.3680085
Epoch : 19 step :  171  Train mse :  0.36705586
Epoch : 19 step :  172  Train mse :  0.36766958
Epoch : 19 step :  173  Train mse :  0.36656582
Epoch : 19 step :  174  Train mse :  0.3678702
Epoch : 19 step :  175  Train mse :  0.3669821
Epoch : 19 step :  176  Train mse :  0.36731294
Epoch : 19 step :  177  Train mse :  0.36707923
Epoch : 19 step :  178  Train mse :  0.36600557
Epoch : 19 step :  179  Train mse :  0.36498633
Epoch : 19 step :  180  Train mse :  0.36395326
Epoch : 19 step :  181  Train mse :  0.36312217
Epoch : 19 step :  182  Train mse :  0.36332712
Epoch : 19 step :  183  Train mse :  0.36321217
Epoch : 19 step :  184  Train mse :  0.3641816
Epoch : 19 step :  185  Train mse :  0.36394098
Epoch : 19 step :  186  Train mse :  0.363032

Epoch : 19 step :  337  Train mse :  0.35986695
Epoch : 19 step :  338  Train mse :  0.3594265
Epoch : 19 step :  339  Train mse :  0.36010528
Epoch : 19 step :  340  Train mse :  0.360331
Epoch : 19 step :  341  Train mse :  0.36074176
Epoch : 19 step :  342  Train mse :  0.36073205
Epoch : 19 step :  343  Train mse :  0.36095992
Epoch : 19 step :  344  Train mse :  0.3607519
Epoch : 19 step :  345  Train mse :  0.3605086
Epoch : 19 step :  346  Train mse :  0.36024067
Epoch : 19 step :  347  Train mse :  0.3606888
Epoch : 19 step :  348  Train mse :  0.36053035
Epoch : 19 step :  349  Train mse :  0.36070743
Epoch : 19 step :  350  Train mse :  0.36172554
Epoch : 19 step :  351  Train mse :  0.36323035
Epoch : 19 step :  352  Train mse :  0.36286035
Epoch : 19 step :  353  Train mse :  0.36249262
Epoch : 19 step :  354  Train mse :  0.3629329
Epoch : 19 step :  355  Train mse :  0.36270466
Epoch : 19 step :  356  Train mse :  0.36252293
Epoch : 19 step :  357  Train mse :  0.3624878


Epoch : 20 step :  152  Train mse :  0.4325969
Epoch : 20 step :  153  Train mse :  0.43456113
Epoch : 20 step :  154  Train mse :  0.43391043
Epoch : 20 step :  155  Train mse :  0.43423986
Epoch : 20 step :  156  Train mse :  0.43425798
Epoch : 20 step :  157  Train mse :  0.43490267
Epoch : 20 step :  158  Train mse :  0.43811694
Epoch : 20 step :  159  Train mse :  0.43765563
Epoch : 20 step :  160  Train mse :  0.43686643
Epoch : 20 step :  161  Train mse :  0.43604928
Epoch : 20 step :  162  Train mse :  0.43740746
Epoch : 20 step :  163  Train mse :  0.43881705
Epoch : 20 step :  164  Train mse :  0.43858564
Epoch : 20 step :  165  Train mse :  0.43757263
Epoch : 20 step :  166  Train mse :  0.43664673
Epoch : 20 step :  167  Train mse :  0.43711984
Epoch : 20 step :  168  Train mse :  0.44713196
Epoch : 20 step :  169  Train mse :  0.44579476
Epoch : 20 step :  170  Train mse :  0.4446308
Epoch : 20 step :  171  Train mse :  0.4455633
Epoch : 20 step :  172  Train mse :  0.4453